In [1]:
import os
os.chdir("/content/drive/MyDrive/Colab Notebooks")

In [ ]:
import numpy as np
import torch
from minitest_quantum.regression_model import *
from minitest_quantum.opt_and_loss import *
from minitest_quantum.torch_early_stopping import  EarlyStopping
from minitest_quantum.train import *
from minitest_quantum.evaluate import *
from tqdm import tqdm
from time import sleep
import math
import random

In [ ]:
npy_path = '/content/drive/MyDrive/Colab Notebooks/datasets/minitest_data'
train_data = ( np.load(npy_path+'/data_train.npy'),
              np.load(npy_path+'/label_train.npy') )

In [ ]:
input, lab = train_data
input, lab = map ( lambda x : torch.from_numpy(x).float(), [input, lab] )

In [ ]:
input.shape

In [ ]:
num_features_input=input.shape[1]
num_features_output=lab.shape[1]
kernel_size=3
#hidden_dim_2=20
dx=input.shape[2]
dy=input.shape[3]

In [ ]:
model= RegressionModel(num_features_input=num_features_input, num_features_output=num_features_output,
                        kernel_size=kernel_size, dx=dx, dy=dy)

In [ ]:
train_set = torch.utils.data.TensorDataset(input, lab)
train_loader = torch.utils.data.DataLoader(train_set, batch_size = 32)

In [ ]:
n_epochs = 200
opt = torch.optim.Adam(model.parameters(), lr=0.008, betas=(0.9, 0.98), eps=1e-9)
schedule=ScheduledOptim(optimizer=opt,n_warmup_steps=30000,initial_lr=0.008,end_lr=0.0001)
early_stopping=EarlyStopping(patience=25, verbose=True)
loss_fun = nn.MSELoss()

In [ ]:
n_batch_validation = int(0.2*len(train_loader))
n_batch_train = len(train_loader) - n_batch_validation

In [ ]:
train_losses, val_losses, train_mape, val_mape = train(model,n_epochs,opt,schedule,n_batch_validation,n_batch_train,train_loader,loss_fun,early_stopping)